<a href="https://colab.research.google.com/github/muajnstu/Multi-Class-Classification-of-YouTube-Videos-Using-A-BERT-enhanced-Machine-Learning-approach/blob/main/XLNet_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade transformers

In [ ]:
!pip cache purge

Files removed: 66


In [ ]:
# Install required packages
!pip install torch transformers datasets scikit-learn --no


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

ambiguous option: --no (--no-binary, --no-build-isolation, --no-cache-dir, --no-clean, --no-color, --no-compile, --no-dependencies, --no-deps, --no-index, --no-input, --no-python-version-warning, --no-use-pep517, --no-user, --no-warn-conflicts, --no-warn-script-location?)


In [ ]:
import re
import string
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import torch
from torch.utils.data import Dataset
from transformers import (
    XLNetTokenizer,
    XLNetForSequenceClassification,
    Trainer,
    TrainingArguments
)

# 1. Data Loading & Preprocessing
df = pd.read_csv('https://raw.githubusercontent.com/muajnstu/ML-Datasets/refs/heads/main/Youtube%20Video%20Dataset.csv')

category_map = {
    "travel blog": 0,
    "science&technology": 1,
    "food": 2,
    "art&music": 3,
    "manufacturing": 4,
    "history": 5
}
df["Category"] = df["Category"].str.lower().str.strip().map(category_map)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
df['processed_text'] = df['Title'].astype(str) + " " + df['Description'].astype(str)

def extract_txt(text):
    match = re.search(r"(?<=\s\-\s).*", str(text))
    return match.group(0) if match else text

df['processed_text'] = df['processed_text'].apply(extract_txt)

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\d', '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['processed_text'] = df['processed_text'].apply(clean_text)
df.rename(columns={'processed_text': 'text'}, inplace=True)

# 2. Train/Val/Test Split
X_train, X_temp, y_train, y_temp = train_test_split(df['text'], df['Category'], test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# 3. Tokenizer & Dataset
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

def encode_texts(tokenizer, texts, max_length=256):
    return tokenizer(
        list(texts),
        padding='max_length',
        truncation=True,
        max_length=max_length,
        return_tensors='pt'
    )

train_encodings = encode_texts(tokenizer, X_train)
val_encodings = encode_texts(tokenizer, X_val)
test_encodings = encode_texts(tokenizer, X_test)

train_labels = torch.tensor(y_train.values)
val_labels = torch.tensor(y_val.values)
test_labels = torch.tensor(y_test.values)

class VideoDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {k: v[idx] for k, v in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = VideoDataset(train_encodings, train_labels)
val_dataset = VideoDataset(val_encodings, val_labels)
test_dataset = VideoDataset(test_encodings, test_labels)

# 4. Model
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=6)

# 5. TrainingArguments & Trainer
training_args = TrainingArguments(
    output_dir="./results_xlnet",
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs_xlnet",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    save_total_limit=1
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# 6. Train & Evaluate
trainer.train()
eval_results = trainer.evaluate()
print("Validation Results:", eval_results)

# 7. Test Set Evaluation
test_preds = trainer.predict(test_dataset)
test_preds_labels = test_preds.predictions.argmax(axis=1)
print("Test Accuracy:", accuracy_score(test_labels, test_preds_labels))
print("\nClassification Report:\n", classification_report(test_labels, test_preds_labels))
print("\nConfusion Matrix:\n", confusion_matrix(test_labels, test_preds_labels))

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: moyajibnea (moyajibnea-noakhali-science-and-technology-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,0.670900,0.500565,0.842997
2,0.432400,0.481284,0.859946
3,0.362000,0.487477,0.871543
4,0.296400,0.486272,0.877788
5,0.253100,0.548202,0.877788


Validation Results: {'eval_loss': 0.4862721562385559, 'eval_accuracy': 0.8777876895628903, 'eval_runtime': 37.979, 'eval_samples_per_second': 29.516, 'eval_steps_per_second': 1.869, 'epoch': 5.0}
Test Accuracy: 0.8707664884135472

Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.79      0.82       204
           1       0.78      0.94      0.85       220
           2       0.91      0.86      0.88       191
           3       0.87      0.89      0.88       173
           4       0.92      0.88      0.90       162
           5       0.95      0.87      0.91       172

    accuracy                           0.87      1122
   macro avg       0.88      0.87      0.87      1122
weighted avg       0.88      0.87      0.87      1122


Confusion Matrix:
 [[162  18   8   9   5   2]
 [  1 206   3   5   2   3]
 [  9   8 164   6   2   2]
 [  7   8   1 154   2   1]
 [  6  11   1   2 142   0]
 [  4  13   3   2   1 149]]
